In [21]:
from neo4j import GraphDatabase
import pandas as pd

# Question 2

On affiche les données pour pouvoir comprendre la structure et pouvoir les diviser dans le graphe

In [22]:
df = pd.read_csv('communes-departement-region.csv')
print(df.head())

  code_commune_INSEE       nom_commune_postal  code_postal  \
0               1001  L ABERGEMENT CLEMENCIAT         1400   
1               1002    L ABERGEMENT DE VAREY         1640   
2               1004        AMBERIEU EN BUGEY         1500   
3               1005      AMBERIEUX EN DOMBES         1330   
4               1006                  AMBLEON         1300   

      libelle_acheminement ligne_5   latitude  longitude  code_commune  \
0  L ABERGEMENT CLEMENCIAT     NaN  46.153426   4.926114           1.0   
1    L ABERGEMENT DE VAREY     NaN  46.009188   5.428017           2.0   
2        AMBERIEU EN BUGEY     NaN  45.960848   5.372926           4.0   
3      AMBERIEUX EN DOMBES     NaN  45.996180   4.912273           5.0   
4                  AMBLEON     NaN  45.749499   5.594320           6.0   

  article            nom_commune      nom_commune_complet code_departement  \
0      L'  Abergement-Clémenciat  L'Abergement-Clémenciat                1   
1      L'    Abergement-de

## Séparation des données

On commence par créer des dataframes distincts pour les différents types de noeuds qu'on va créer

### Régions

In [23]:
df_regions = df[['nom_region','code_region']].drop_duplicates().reset_index(drop=True)
df_regions  = df_regions.dropna()

print(df_regions)

                    nom_region  code_region
0         Auvergne-Rhône-Alpes         84.0
1              Hauts-de-France         32.0
2   Provence-Alpes-Côte d'Azur         93.0
3                    Grand Est         44.0
4                    Occitanie         76.0
5                    Normandie         28.0
6           Nouvelle-Aquitaine         75.0
7          Centre-Val de Loire         24.0
8      Bourgogne-Franche-Comté         27.0
9                     Bretagne         53.0
10                       Corse         94.0
11            Pays de la Loire         52.0
12               Île-de-France         11.0
13                  Guadeloupe          1.0
14                  Martinique          2.0
15                      Guyane          3.0
16                  La Réunion          4.0
18                     Mayotte          6.0


### Départements

On garde bien le code de la région auquel il est associé pour pouvoir relier les régions et les départements

In [24]:
df_dep = df[['nom_departement','code_departement', 'code_region']].drop_duplicates().reset_index(drop=True)
df_dep  = df_dep.dropna()

print(df_dep)

             nom_departement code_departement  code_region
0                        Ain                1         84.0
1                      Aisne                2         32.0
2                     Allier                3         84.0
3    Alpes-de-Haute-Provence                4         93.0
4               Hautes-Alpes                5         93.0
..                       ...              ...          ...
96                Guadeloupe              971          1.0
97                Martinique              972          2.0
98                    Guyane              973          3.0
99                La Réunion              974          4.0
101                  Mayotte              976          6.0

[101 rows x 3 columns]


### Communes 

De même, on garde le code du département

In [25]:
df_com = df[['nom_commune','code_commune', 'code_departement']].drop_duplicates().reset_index(drop=True)
df_com  = df_com.dropna()

print(df_com)

                 nom_commune  code_commune code_departement
0      Abergement-Clémenciat           1.0                1
1        Abergement-de-Varey           2.0                1
2          Ambérieu-en-Bugey           4.0                1
3        Ambérieux-en-Dombes           5.0                1
4                    Ambléon           6.0                1
...                      ...           ...              ...
36008                    Voh         831.0               98
36009                   Yate         832.0               98
36010                Kouaoua         833.0               98
36011      Ile de clipperton         901.0               98
36012                 Monaco         138.0               99

[36010 rows x 3 columns]


## Création du graphe

### Connexion au serveur neo4j

In [26]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"))

### Remise à zéro

In [27]:
with driver.session() as session:
    session.run("MATCH (n:region) DETACH DELETE n")
    session.run("MATCH (n:departement) DETACH DELETE n")
    session.run("MATCH (n:commune) DETACH DELETE n")
    session.run("MATCH (n:region_v2) DETACH DELETE n")
    session.run("MATCH (n:departement_v2) DETACH DELETE n")
    session.run("MATCH (n:commune_v2) DETACH DELETE n")
    


### Création des noeuds

In [28]:
with driver.session() as session:
    #Création des régions
    for id, row in df_regions.iterrows() :
        session.run("CREATE (n:region {name: $name, code: $code})", name=row['nom_region'], code=row['code_region'])
    
    #Création des départements
    for id, row in df_dep.iterrows() :
        session.run("CREATE (n:departement {name: $name, code: $code, region: $region})", name=row['nom_departement'], code=row['code_departement'], region=row['code_region'])
    
    #Création des communes
    for id, row in df_com.iterrows() :
                session.run("CREATE (n:departement {name: $name, code: $code, departement: $region})", name=row['nom_commune'], code=row['code_commune'], region=row['code_departement'])


### Création des relations

In [29]:
with driver.session() as session:
    # liaison région/département
    session.run("MATCH (r:region),(d:departement) WHERE r.code = d.region_code CREATE (r)-[A:APPARTIENT_A]->(d)")
    
    #liaison département/commune
    session.run("MATCH (d:departement),(c:commune) WHERE d.code = c.departement_code CREATE (d)-[A:APPARTIENT_A]->(c)")